In [40]:
#importing all libraries
import numpy as np
np.random.seed(1)
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objs as go
#import altair as alt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import *

from fbprophet import Prophet
from sklearn.ensemble import IsolationForest

In [41]:
#reading the dataset
data = pd.read_csv("/training_1416 (1).csv")
y = data['nttack'] #storing the ground truth value
data.head()

,Unnamed: 0,rcvTime,sendTime,sender,senderPseudo,messageID,pos_x,pos_y,spd_x,spd_y,acl_x,acl_y,hed_x,hed_y,receiver_id,nttack
0,0,50408.087237,50408.087237,27.0,10275.0,16969.0,223.005975,364.802772,-0.079871,0.537359,-0.388361,2.612909,-0.14702,0.989133,21,0.0
1,1,50408.392900,50408.392900,21.0,10215.0,17039.0,220.358432,383.935288,-0.078168,0.525905,-0.388913,2.616645,-0.14702,0.989133,27,0.0
2,2,50409.087237,50409.087237,27.0,10275.0,17121.0,222.757252,366.657904,-0.419586,2.822917,-0.360449,2.425112,-0.14702,0.989133,21,0.0
3,3,50409.392900,50409.392900,21.0,10215.0,17151.0,220.152543,385.761996,-0.421901,2.838498,-0.322463,2.169573,-0.14702,0.989133,27,0.0
4,4,50410.087237,50410.087237,27.0,10275.0,17193.0,222.453841,370.546522,-0.695096,4.676514,-0.330370,2.222776,-0.14144,0.989947,21,0.0


In [42]:
#information about the variables in the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130329 entries, 0 to 1130328
Data columns (total 16 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Unnamed: 0    1130329 non-null  int64  
 1   rcvTime       1130329 non-null  float64
 2   sendTime      1130329 non-null  float64
 3   sender        1130329 non-null  float64
 4   senderPseudo  1130329 non-null  float64
 5   messageID     1130329 non-null  float64
 6   pos_x         1130329 non-null  float64
 7   pos_y         1130329 non-null  float64
 8   spd_x         1130329 non-null  float64
 9   spd_y         1130329 non-null  float64
 10  acl_x         1130329 non-null  float64
 11  acl_y         1130329 non-null  float64
 12  hed_x         1130329 non-null  float64
 13  hed_y         1130329 non-null  float64
 14  receiver_id   1130329 non-null  int64  
 15  nttack        1130329 non-null  float64
dtypes: float64(14), int64(2)
memory usage: 138.0 MB


In [43]:
#converting receiver time coloumn to date time format
from datetime import datetime

data['Date Time']= pd.to_datetime(data['rcvTime'], unit='s')

In [44]:
data['Date Time']

0         1970-01-01 14:00:08.087237064
1         1970-01-01 14:00:08.392900129
2         1970-01-01 14:00:09.087237193
3         1970-01-01 14:00:09.392900257
4         1970-01-01 14:00:10.087237320
                       ...             
1130324   1970-01-01 15:27:26.969135891
1130325   1970-01-01 15:27:26.969135897
1130326   1970-01-01 15:27:26.969135898
1130327   1970-01-01 15:27:26.969136009
1130328   1970-01-01 15:27:26.969136052
Name: Date Time, Length: 1130329, dtype: datetime64[ns]

In [45]:
#reviewing the datatypes
print(data.dtypes)

Unnamed: 0               int64
rcvTime                float64
sendTime               float64
sender                 float64
senderPseudo           float64
messageID              float64
pos_x                  float64
pos_y                  float64
spd_x                  float64
spd_y                  float64
acl_x                  float64
acl_y                  float64
hed_x                  float64
hed_y                  float64
receiver_id              int64
nttack                 float64
Date Time       datetime64[ns]
dtype: object


In [46]:
#dropping not required coloumns
data.drop(['Unnamed: 0','rcvTime','sendTime','nttack'],inplace=True,axis=1)

In [48]:
#sorted the values based on date time and putting the date time coloumn to the index
x = data.sort_values(by=['Date Time']).reset_index(drop=True)
x.set_index('Date Time',inplace=True)
x

,sender,senderPseudo,messageID,pos_x,pos_y,spd_x,spd_y,acl_x,acl_y,hed_x,hed_y,receiver_id
Date Time,,,,,,,,,,,,
1970-01-01 14:00:08.087237064,27.0,10275.0,16969.0,223.005975,364.802772,-0.079871,0.537359,-0.388361,2.612909,-0.147020,0.989133,21
1970-01-01 14:00:08.392900129,21.0,10215.0,17039.0,220.358432,383.935288,-0.078168,0.525905,-0.388913,2.616645,-0.147020,0.989133,27
1970-01-01 14:00:09.087237193,27.0,10275.0,17121.0,222.757252,366.657904,-0.419586,2.822917,-0.360449,2.425112,-0.147020,0.989133,21
1970-01-01 14:00:09.392900257,21.0,10215.0,17151.0,220.152543,385.761996,-0.421901,2.838498,-0.322463,2.169573,-0.147020,0.989133,27
1970-01-01 14:00:10.087237320,27.0,10275.0,17193.0,222.453841,370.546522,-0.695096,4.676514,-0.330370,2.222776,-0.141440,0.989947,21
...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 15:27:26.969135891,7335.0,2073355.0,22841012.0,226.535204,219.450184,2.866232,0.415503,2.982118,0.336361,0.999531,0.030620,7701
1970-01-01 15:27:26.969135897,7335.0,2073355.0,22841012.0,226.535204,219.450184,2.866232,0.415503,2.982118,0.336361,0.999531,0.030620,7617
1970-01-01 15:27:26.969135898,7335.0,2073355.0,22841012.0,226.535204,219.450184,2.866232,0.415503,2.982118,0.336361,0.999531,0.030620,7497


### Define and fit the model:

In [49]:
#creating and fitting the model
model=IsolationForest(n_estimators=100)

model.fit(x)

print(model.get_params())

{'bootstrap': False, 'contamination': 'auto', 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [53]:
#predicting the model and checking the metrics
ypred = model.predict(x)
print(f'Accuracy Score{accuracy_score(y,ypred)}')
print(f'Cohen Kappa Score{cohen_kappa_score(y,ypred)}')
print(f'Mathews Score{matthews_corrcoef(y,ypred)}')

Accuracy Score0.46326600485345415
Cohen Kappa Score0.028321509342995044
Mathews Score0.03698820700293777


# Local Outlier Factor




In [70]:
from sklearn.neighbors import LocalOutlierFactor


In [73]:
#initializing local outlier factor model and checking the performance
clf = LocalOutlierFactor()
ypred = clf.fit_predict(x)
print(f'Accuracy Score: {accuracy_score(y,ypred)}')
print(f'Cohen Kappa Score: {cohen_kappa_score(y,ypred)}')
print(f'Mathews Score: {matthews_corrcoef(y,ypred)}')

Accuracy Score: 0.4663766036260239
Cohen Kappa Score: -0.002425491390083856
Mathews Score: -0.003215758716578438
